In [1]:
!pip install tsfresh

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.8/95.8 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.1/169.1 kB 9.2 MB/s eta 0:00:00


In [2]:
# This notebook is used to generate the data file "work/Data/processed/Cow_Prob_dataset_L1.csv" 
# for Exp_1 and Exp_2 based on Lactation period 1. The actual analyses are in 

# work/HoldingPenTime/GP_Model/Cow_Detection_L1_Exp_1 .ipynb

# work/HoldingPenTime/GP_Model/Early_Cow_Detection_TS_length_early_150_days_Exp_2.ipynb
# work/HoldingPenTime/GP_Model/Early_Cow_Detection_TS_length_last_150_days_Exp_2.ipynb


In [1]:
'''
#Gigacow-tools# - data collection for fast/slow learner.
This scripts used for single cow data collection work.
Data Tables: gigacow_filter.csv, lactation_filter.csv, traffic_raw_filter.csv
'''

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
pd.options.mode.chained_assignment = None
from tsfresh import extract_features
from tsfresh import select_features
from tsfresh.utilities.dataframe_functions import impute
from tsfresh.feature_selection.relevance import calculate_relevance_table
from tsfresh.feature_extraction import ComprehensiveFCParameters, MinimalFCParameters
from sklearn.preprocessing import StandardScaler, OneHotEncoder

#access data from local directory
dataDir = Path.cwd().parent.parent/'Data/processed'
gigacow_cols = ['Gigacow_Cow_Id', 'FarmName_Pseudo', 'BreedName', 'BirthDate']
lactation_cols = ['Gigacow_Cow_Id', 'FarmName_Pseudo', 'LactationInfoDate', 'LactationNumber', 'DaysInMilk']
gigacow = pd.read_csv(dataDir/'gigacow_filter.csv', encoding='utf-8', usecols=gigacow_cols)
lactation = pd.read_csv(dataDir/'lactation_filter.csv', encoding='utf-8', usecols=lactation_cols)
traffic = pd.read_csv(dataDir/'traffic_raw_filter.csv', encoding='utf-8', index_col=False)
#check out cows with most milking events
print(traffic.Gigacow_Cow_Id.value_counts().nlargest(10))

5613    17247
4504    16330
5046    15623
5147    15440
3147    15303
478     14751
6380    14514
1181    14434
4478    14434
1985    14419
Name: Gigacow_Cow_Id, dtype: int64


In [4]:
# Select cows with sufficient data points on single lactation periods
# Try to collect cow's data that contain milking events on lactation periods 1
# fetch all milking traffic events for merging
traffic_milking = traffic.TrafficResult.str.contains('kg', regex=False)
all_list = traffic_milking.index[traffic_milking.values == True].tolist()
milking_total = traffic[traffic.index.isin(all_list)]
milking_total.TrafficEventDateTime = pd.to_datetime(milking_total.TrafficEventDateTime)
milking_total['milking_date'] = milking_total.TrafficEventDateTime.dt.date

# convert data type
milking_total.milking_date = pd.to_datetime(milking_total.milking_date)
lactation.LactationInfoDate = pd.to_datetime(lactation.LactationInfoDate)
# merge all milking events with lactation table for filtering
milking_total = milking_total.merge(lactation, how='left', left_on=['FarmName_Pseudo', 'Gigacow_Cow_Id', 'milking_date'], right_on=['FarmName_Pseudo', 'Gigacow_Cow_Id', 'LactationInfoDate'])
def lac_collect(NumLac, milking_total):
    """Generate cow list for multiple lactation periods.

    Args:
        NumLac: The number of lactaion period
        milking_total: A dataframe contains all the milkings events

    Returns:
        A list contain all the cows events with sufficient data points within the lactation periods.
    """
    cow_list = list()
    for num in range(1, NumLac+1):
        # select records that contains lactation period #num
        milking_select1 = milking_total.loc[milking_total['LactationNumber'] == num]
        milking_select1.drop_duplicates(subset=['Gigacow_Cow_Id', 'milking_date', 'LactationNumber', 'DaysInMilk'], inplace=True)
        # drop the anomaly data point
        milking_select1 = milking_select1.loc[milking_select1.DaysInMilk < 400]
        # select sufficient data points on lactation
        selected1 = milking_select1.Gigacow_Cow_Id.value_counts(ascending=True)
        selected1 = selected1.loc[(selected1.values > 150) & (selected1.values < 365)]
        selected_cow_list = selected1.index.to_list()
        if num == 1:
            cow_list = selected_cow_list
        cow_list = list(set(cow_list) & set(selected_cow_list))
        print(len(cow_list), cow_list)
    return cow_list

cow_list = lac_collect(1, milking_total)

147 [2560, 3075, 2568, 2569, 4109, 3603, 1555, 6164, 5147, 3613, 3104, 1057, 544, 550, 5160, 2090, 4142, 48, 3638, 5691, 2112, 3657, 74, 3149, 6222, 1103, 4176, 2135, 1113, 6235, 603, 4704, 6241, 4195, 5225, 4714, 622, 5235, 5239, 3705, 4730, 3707, 645, 3206, 5767, 2694, 3718, 5262, 2702, 151, 6300, 4765, 1181, 5279, 3744, 4769, 2724, 1705, 6316, 3762, 5301, 182, 2742, 4280, 4792, 708, 4295, 5322, 2251, 1738, 1744, 5846, 4311, 1755, 1244, 5852, 2278, 2792, 6380, 3822, 5873, 2805, 5372, 1277, 769, 1796, 4871, 2844, 289, 6434, 4899, 6439, 4910, 1842, 1331, 6452, 2868, 3890, 1336, 6463, 2374, 6472, 4939, 2386, 855, 5465, 1905, 3446, 2423, 3447, 1910, 4990, 6018, 2956, 6039, 2460, 926, 3487, 5534, 2466, 2979, 4524, 4525, 4012, 943, 5555, 5046, 951, 1464, 4537, 1985, 961, 6086, 2510, 978, 5076, 2517, 985, 478, 5088, 5092, 3045, 2031, 5616, 6127, 2039, 2047]


In [5]:
def countCowAge(birthDate, milkingDate):
    '''
    func: Calculate cows age based on birthDate and milkingDate
    args: 
        birthDate: cow's birth datetime
        milkingDate: milking events datetime
    return: cow age in human years(float)
    '''
    birthDate = pd.to_datetime(birthDate)
    milkingDate = pd.to_datetime(milkingDate)
    days = np.float32(np.datetime64(milkingDate, 'D') - np.datetime64(birthDate, 'D'))
    age = np.around(days/365, 2)
    return age

# select single cow from the traffic table

def data_collector(traffic, gigacow, lactation, cow_id, lacNumList):
    '''
    func: collect features from for a single cow
    args: 
        traffic: traffic data table
        gigacow: gigacow data table
        lactation: lactation data table
        cow_id: gigacow_id of the cow
        lacNumList: a list lactation period number
    return: A dataframe contains all features for a single cow on specfic lactation period
    '''

    traffic_single_cow = traffic.loc[traffic['Gigacow_Cow_Id'] == cow_id]
    traffic_single_cow.sort_values(by='TrafficEventDateTime', inplace=True)
    traffic_single_cow.index = range(len(traffic_single_cow))

    '''
        Extract Milking Event and its most recent traffic event to calculate T2-T1
        T1: Entry time into the Mjolkfalla
        T2: Entry time into the milking robot
        T2-T1: calculate time difference between T2&T1 (i.e., Time spend in Mjolkfalla/holding area)
    '''
    # locate mikling event by searching 'kg' keyword in traffic result
    # the most recent traffic event to milking event should be pre_milking event
    # need to filter out records with gate failure
    track_milking = traffic_single_cow.TrafficResult.str.contains('kg', regex=False)
    milking_index_list = track_milking.index[track_milking.values == True].tolist()
    pre_milking_index_list = [x-1 for x in milking_index_list]
    milking_traffic = traffic_single_cow[traffic_single_cow.index.isin(milking_index_list)]
    pre_milking_traffic = traffic_single_cow[traffic_single_cow.index.isin(pre_milking_index_list)]

    # drop rows that the gate failed to detect cows but have milking result
    # previous area in milking_traffic table should only be Mjolkfalla
    # previous area in pre_milking_traffic table should not be Mjolkfalla
    failed_list_1_milk = milking_traffic.index[milking_traffic['PreviousArea'] == 'Koridor till Sorteringsgrind 2'].tolist()
    failed_list_1_pre = [x-1 for x in failed_list_1_milk]
    failed_list_2_pre = pre_milking_traffic.index[pre_milking_traffic['PreviousArea'] == 'Mjolkfalla'].tolist()
    failed_list_2_milk = [x+1 for x in failed_list_2_pre]
    # traffic result in pre_milking_traffic table should contain Mjolkfalla
    track_pre_milking = pre_milking_traffic.TrafficResult.str.contains('Mjolkfalla', regex=False)
    failed_list_3_pre = track_pre_milking.index[track_pre_milking.values == False].tolist()
    failed_list_3_milk = [x+1 for x in failed_list_3_pre]

    # remove failed records based on index list
    milking_traffic_failed = failed_list_1_milk + failed_list_2_milk + failed_list_3_milk
    pre_milking_traffic_failed = failed_list_1_pre + failed_list_2_pre + failed_list_3_pre
    milking_traffic.drop(axis=0, index=milking_traffic_failed, inplace=True)
    pre_milking_traffic.drop(axis=0, index=pre_milking_traffic_failed, inplace=True)
    # concatenate two tables to track the traffic directly
    all_milking_traffic = pd.concat([milking_traffic, pre_milking_traffic])
    all_milking_traffic.sort_values(by=['TrafficEventDateTime'], inplace=True)
    #rename table columns for merging
    milking_traffic.rename(columns={"TrafficEventDateTime": "MilkingEventDateTime", "TrafficResult": "MilkProduction", "TimeInArea_totalSeconds": "RoundedSecondsTimeInArea"}, inplace=True)
    pre_milking_traffic.rename(columns={"TrafficEventDateTime": "Pre_MilkingEventDateTime", "TimeInArea_totalSeconds": "RoundedSecondsTimeInArea"}, inplace=True)
    # unify the index of two tables
    milking_traffic.index = range(len(milking_traffic))
    pre_milking_traffic.index = range(len(pre_milking_traffic))
    # inert "pre_traffic_milking" to milking traffic table
    milking_traffic.insert(5, 'Pre_MilkingEventDateTime', pre_milking_traffic['Pre_MilkingEventDateTime'])
    # calculate T2-T1
    milking_traffic.MilkingEventDateTime = pd.to_datetime(milking_traffic.MilkingEventDateTime)
    milking_traffic.Pre_MilkingEventDateTime = pd.to_datetime(milking_traffic.Pre_MilkingEventDateTime)
    milking_traffic['timeDelta_Seconds'] = (milking_traffic['MilkingEventDateTime'] - milking_traffic['Pre_MilkingEventDateTime']).dt.total_seconds()

    # extract traffic result(milk production)
    milking_traffic['MilkProduction'].replace(r"[^0-9.,]+"," ", inplace=True, regex=True)
    milking_traffic['MilkProduction'].replace(r"\s*","", inplace=True, regex=True)
    milking_traffic['MilkProduction'].replace(r"[,]+",".", inplace=True, regex=True)
    milking_traffic['MilkProduction'] = milking_traffic['MilkProduction'].astype('float64')

    # merge all the other features into milking_traffic table
    milking_traffic['MilkingDate'] = milking_traffic.MilkingEventDateTime.dt.date
    milking_traffic.MilkingDate = pd.to_datetime(milking_traffic.MilkingDate)
    lactation.LactationInfoDate = pd.to_datetime(lactation.LactationInfoDate)
    single_cow_merge = milking_traffic.merge(lactation, how='left', left_on=['FarmName_Pseudo', 'Gigacow_Cow_Id', 'MilkingDate'], right_on=['FarmName_Pseudo', 'Gigacow_Cow_Id', 'LactationInfoDate'])
    single_cow_merge = single_cow_merge.merge(gigacow, how='left', left_on=['FarmName_Pseudo', 'Gigacow_Cow_Id'], right_on=['FarmName_Pseudo', 'Gigacow_Cow_Id'])

    # drop failed data points based on RoundedSecondsTimeInArea & timeDelta_Seconds
    single_cow_merge.drop(single_cow_merge.loc[abs(single_cow_merge.timeDelta_Seconds - single_cow_merge.RoundedSecondsTimeInArea) > 300].index, inplace=True)
    single_cow_merge['TrafficDeviceName'].replace(r"[A-Za-z]+\s*","vms", inplace=True, regex=True)
    # calculate age of cows
    single_cow_merge['Age'] = single_cow_merge.apply(lambda x: countCowAge(x['BirthDate'], x['MilkingEventDateTime']), axis=1)
    single_cow_merge.drop(['BirthDate'], axis=1, inplace=True)
    single_cow_merge.dropna(inplace=True)

    # integrate multiple milking events for a single DIM
    single_cow_merge = single_cow_merge[single_cow_merge.LactationNumber.isin(lacNumList)]
    single_cow_merge.index = range(1,len(single_cow_merge)+1) 
    single_cow_merge.drop(['MilkingEventDateTime', 'Pre_MilkingEventDateTime', 'Traffic_Id', 'MilkingInterval_totalSeconds', 'RoundedSecondsTimeInArea', 'PreviousArea', 'GroupName', 'LactationInfoDate', 'TrafficDeviceName'], axis=1, inplace=True)

    # uncomment following part to get combined milking events for each DIM
    # comb_cows = single_cow_merge.groupby(by=['FarmName_Pseudo', 'Gigacow_Cow_Id', 'MilkingDate', 'LactationNumber', 'DaysInMilk', 'BreedName', 'Age'], sort=False, as_index=False).sum(['MilkProduction', 'timeDelta_Seconds'])
    # single_cow_merge_size = single_cow_merge.groupby(by=['FarmName_Pseudo', 'Gigacow_Cow_Id', 'MilkingDate', 'LactationNumber', 'DaysInMilk', 'BreedName', 'Age'], sort=False, as_index=False).size()
    # comb_cows = pd.concat([comb_cows, single_cow_merge_size['size']], axis=1, ignore_index=False)
    # comb_cows.rename(columns={"MilkProduction": "Total_MilkProduction", "timeDelta_Seconds": "Total_timeDelta_Seconds", "size": "milking_times"}, inplace=True)
    # comb_cows.index = range(1, len(comb_cows)+1)
    # return comb_cows

    single_cow_merge.rename(columns={"MilkProduction": "Total_MilkProduction", "timeDelta_Seconds": "Total_timeDelta_Seconds", "size": "milking_times"}, inplace=True)
    single_cow_merge.index = range(1, len(single_cow_merge)+1)
    return single_cow_merge


In [6]:
"""
labeling cow with problematic/normal(1/0)
"""
threshold_ratio = 0.05
Path(dataDir/'Problematic_targetCows').mkdir(parents=True, exist_ok=True)
def labeling_problematic(threshold_ratio, cow_total): 
    '''
    func: labeling problematic cows
    args: 
        threshold_ratio: threshold ratio for the abnormal event milking events
        cow_total: A dataframe contains all data points for a single cow
    return: problematic cows dataset with label
    '''
    global learner
    total_events = len(cow_total)
    abnoramal_cows = cow_total.loc[cow_total.Total_timeDelta_Seconds > 7200]
    abnoramal_ratio = len(abnoramal_cows)/total_events
    print(abnoramal_ratio)
    if abnoramal_ratio > threshold_ratio:
        problematic = 1 # problematic cow
    else:
        problematic = 0 # normal cow
    cow_total['problematic'] = problematic
    return cow_total

In [7]:
# filter out cows' record start at the middle of the lactation
filter_list = []
for id in cow_list:
    single_cow = data_collector(traffic, gigacow, lactation, id, [1])
    if single_cow.DaysInMilk.min() < 60:
        filter_list.append(id)

print("filtered: ", len(filter_list), filter_list)

filtered:  142 [2560, 3075, 2568, 2569, 4109, 3603, 1555, 6164, 5147, 3613, 3104, 1057, 544, 550, 5160, 2090, 4142, 48, 3638, 5691, 2112, 3657, 74, 3149, 6222, 1103, 4176, 2135, 1113, 6235, 603, 4704, 6241, 4195, 5225, 4714, 5235, 5239, 3705, 4730, 3707, 645, 3206, 5767, 2694, 3718, 5262, 2702, 151, 6300, 4765, 1181, 5279, 3744, 4769, 2724, 1705, 6316, 5301, 2742, 4280, 4792, 708, 4295, 5322, 2251, 1738, 1744, 5846, 4311, 1755, 1244, 5852, 2278, 2792, 6380, 3822, 5873, 2805, 5372, 1277, 769, 1796, 4871, 2844, 289, 6434, 4899, 6439, 4910, 1842, 1331, 6452, 2868, 3890, 1336, 6463, 2374, 6472, 4939, 2386, 855, 5465, 1905, 3446, 2423, 3447, 1910, 4990, 6018, 2956, 6039, 2460, 926, 5534, 2466, 2979, 4524, 4525, 4012, 943, 5555, 5046, 951, 1464, 1985, 961, 6086, 2510, 978, 5076, 2517, 985, 478, 5088, 5092, 3045, 2031, 5616, 6127, 2039, 2047]


In [8]:
#Uses the labeling_problematic function to label problematic cows and print distribution of time into a PDF
from matplotlib.backends.backend_pdf import PdfPages
import warnings
pd.set_option('mode.chained_assignment', None)

""" plot the relations between timeDelta and Lactation/DIM(DaysInMilk)
        @@@ Total_timeDelta @@@
    """ 
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    with PdfPages(dataDir/'100cows_timecost_scatters_lac1_with_label_scale30000.pdf') as pdf:
        for id in filter_list:
            print("cow_id:", id)
            single_cow_merge = data_collector(traffic, gigacow, lactation, id, [1])
            single_cow_merge = labeling_problematic(0.05, single_cow_merge)
            prob = single_cow_merge.problematic.unique()[0]
            fig1 = plt.figure()
            # fig2 = plt.figure()
            if prob == 1:
                title = "Problematic_Cow_cow_id_"+ str(id)
            else:
                title = "Normal_Cow_cow_id_"+ str(id)
            fig1 = single_cow_merge.loc[single_cow_merge.LactationNumber == 1].plot(x="DaysInMilk", y="Total_timeDelta_Seconds", kind='scatter', title=title+"_Lac1", xlim=[1, 360], ylim=[0, 30000], s=2, c='b')
            # fig2 = single_cow_merge.loc[single_cow_merge.LactationNumber == 2].plot(x="DaysInMilk", y="Total_timeDelta_Seconds", kind='scatter', title=title+"_Lac2", xlim=[1, 360], ylim=[0, 10000], s=2, c='b')
            pdf.savefig(fig1.get_figure())
            # pdf.savefig(fig2.get_figure())
            plt.close()

cow_id: 2560
0.003472222222222222
cow_id: 3075
0.0680379746835443
cow_id: 2568
0.14465408805031446
cow_id: 2569
0.0
cow_id: 4109
0.24615384615384617
cow_id: 3603
0.032362459546925564
cow_id: 1555
0.0017452006980802793
cow_id: 6164
0.17270788912579957
cow_id: 5147
0.0
cow_id: 3613
0.0
cow_id: 3104
0.01238390092879257
cow_id: 1057
0.01703800786369594
cow_id: 544
0.21496815286624205
cow_id: 550
0.05955678670360111
cow_id: 5160
0.004722550177095631
cow_id: 2090
0.003260869565217391
cow_id: 4142
0.09314586994727592
cow_id: 48
0.018907563025210083
cow_id: 3638
0.0027397260273972603
cow_id: 5691
0.4318181818181818
cow_id: 2112
0.013793103448275862
cow_id: 3657
0.09400544959128065
cow_id: 74
0.0026490066225165563
cow_id: 3149
0.017467248908296942
cow_id: 6222
0.039540816326530615
cow_id: 1103
0.0020408163265306124
cow_id: 4176
0.032577903682719546
cow_id: 2135
0.06801736613603473
cow_id: 1113
0.0074005550416281225
cow_id: 6235
0.27607361963190186
cow_id: 603
0.009497964721845319
cow_id: 4704
0

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

...

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

In [9]:
pd.options.mode.chained_assignment = None

mean_totalTimeCost = 0
Path(dataDir/'Problematic_targetCows').mkdir(parents=True, exist_ok=True)
lactationNum = [1]

# save a list of cow data for abnormal cows detection problem
for i, cow_id in enumerate(filter_list):
    single_cow_merge = data_collector(traffic, gigacow, lactation, cow_id, lactationNum)
    mean_totalTimeCost += single_cow_merge.Total_timeDelta_Seconds.mean()
    single_cow_merge = labeling_problematic(threshold_ratio, single_cow_merge)
    problematic = single_cow_merge.problematic.unique()[0]
    if problematic == 1:
        print("This cow is problematic")
    single_cow_merge["id"] = i+1
    single_cow_merge.dropna(inplace=True)
    fileName = 'Problematic_targetCows/cow_' + str(i) + '.csv'
    single_cow_merge.to_csv(dataDir/fileName)
print("num of cows: ", len(cow_list))
print("Mean of total time cost: ", mean_totalTimeCost/len(cow_list))

0.003472222222222222
0.0680379746835443
This cow is problematic
0.14465408805031446
This cow is problematic
0.0
0.24615384615384617
This cow is problematic
0.032362459546925564
0.0017452006980802793
0.17270788912579957
This cow is problematic
0.0
0.0
0.01238390092879257
0.01703800786369594
0.21496815286624205
This cow is problematic
0.05955678670360111
This cow is problematic
0.004722550177095631
0.003260869565217391
0.09314586994727592
This cow is problematic
0.018907563025210083
0.0027397260273972603
0.4318181818181818
This cow is problematic
0.013793103448275862
0.09400544959128065
This cow is problematic
0.0026490066225165563
0.017467248908296942
0.039540816326530615
0.0020408163265306124
0.032577903682719546
0.06801736613603473
This cow is problematic
0.0074005550416281225
0.27607361963190186
This cow is problematic
0.009497964721845319
0.03292181069958848
0.014245014245014245
0.00922266139657444
0.002044989775051125
0.03800475059382423
0.017857142857142856
0.02361111111111111
0.0

In [12]:
""" Data Preparation L1 """
cow_total = None
usecols = ['id', 'FarmName_Pseudo', 'Gigacow_Cow_Id', 'Total_MilkProduction', 'Total_timeDelta_Seconds', 'LactationNumber', 'DaysInMilk', 'BreedName', 'Age', 'MilkingDate', 'problematic']
dataDir = Path.cwd().parent.parent/'Data/processed/Problematic_targetCows/'

# integrate all the cows data into one dataset
filelist = list(Path(dataDir).glob('cow_*.csv'))
for i, _ in enumerate(filelist):
    fileName = 'cow_' + str(i) + '.csv'
    single_cow = pd.read_csv(dataDir/fileName, encoding='utf-8', usecols=usecols)
    single_cow.sort_values(by=['MilkingDate'], inplace=True)
    if i == 0:
        cow_total = single_cow
    else:
        cow_total = pd.concat([cow_total, single_cow], axis=0, ignore_index=True)
cow_total.to_csv(dataDir.parent/"Cow_Prob_dataset_L1.csv", index=False)
cow_total

,FarmName_Pseudo,Gigacow_Cow_Id,Total_MilkProduction,Total_timeDelta_Seconds,MilkingDate,LactationNumber,DaysInMilk,BreedName,Age,problematic,id
0,a624fb9a,2560,9.38,3176.0,2022-02-14,1.0,2.0,1,2.15,0,1
1,a624fb9a,2560,8.46,352.0,2022-02-14,1.0,2.0,1,2.15,0,1
2,a624fb9a,2560,6.68,997.0,2022-02-15,1.0,3.0,1,2.15,0,1
3,a624fb9a,2560,7.34,9274.0,2022-02-15,1.0,3.0,1,2.15,0,1
4,a624fb9a,2560,8.15,407.0,2022-02-16,1.0,4.0,1,2.15,0,1
...,...,...,...,...,...,...,...,...,...,...,...
95849,a624fb9a,2047,7.96,59.0,2022-11-12,1.0,322.0,1,3.02,0,142
95850,a624fb9a,2047,5.53,148.0,2022-11-13,1.0,323.0,1,3.03,0,142
95851,a624fb9a,2047,3.24,287.0,2022-11-13,1.0,323.0,1,3.03,0,142
95852,a624fb9a,2047,9.23,240.0,2022-11-13,1.0,323.0,1,3.03,0,142
